In [ ]:
# Handle the auto_batch_size runs logging 
# Feature Comparison                                       (average over models)
# Model Comparison                                         (average over features)
# Stratification index comparison                          (just for Distilbert)
# Warmup steps Comparison                                  (average over features)

# Best set for distilbert and best set overall
# Model selection comparisons                              (just pearson score)
# Kaggle test notebook                                     (Just for selected model)

# Performance plot (training time vs pearson score)        (Just for best feature)
# Performance plot (number of parameters vs pearson score) (Just for best feature)

In [57]:
import os
import re
import json
import pandas as pd
from matplotlib import pyplot as plt

In [58]:
run_paths = []
log_dir = "ray_results"
model_names = ["distilbert-base-uncased","AI-Growth-Lab/PatentSBERTa"]
stratification_fields = ["score_map", "stratification_index"]

for model in model_names:
    source_dir = f"../../ray_results/{model}"

    model_paths = [p for p in os.listdir(source_dir) if "trainable_" in p]
    run_paths.extend(model_paths)

for path in run_paths:
    print(path)

trainable_dcb85_00000_0_features=anchor_target_CPCdescription,model=distilbert-base-uncased,stratify_on=stratification_index_2023-02-02_14-38-24
trainable_dcb85_00004_4_features=anchor_target_CPCdescription,model=distilbert-base-uncased,stratify_on=score_map_2023-02-02_18-13-34
trainable_dcb85_00005_5_features=same_anchor_similar_targets,model=distilbert-base-uncased,stratify_on=score_map_2023-02-02_18-52-57
trainable_dcb85_00002_2_features=same_anchor_context_similar_targets,model=distilbert-base-uncased,stratify_on=stratification_inde_2023-02-02_16-20-18
trainable_dcb85_00001_1_features=same_anchor_similar_targets,model=distilbert-base-uncased,stratify_on=stratification_index_2023-02-02_15-18-10
trainable_927e3_00000_0_features=CPCdescription_same_anchor_context_similar_targets,model=distilbert-base-uncased,stratify_on=scor_2023-02-03_17-19-47
trainable_dcb85_00003_3_features=CPCdescription_same_anchor_context_similar_targets,model=distilbert-base-uncased,stratify_on=stra_2023-02-02_

In [60]:
df_list = []
for model in model_names:
    source_dir = f"../../ray_results/{model}"
    model_paths = [p for p in os.listdir(source_dir) if "trainable_" in p]

    for p in model_paths:
        df = pd.read_csv(os.path.join(source_dir, p, "progress.csv"))

        with open(os.path.join(source_dir,p,"params.json")) as f:
            params = json.load(f)
        features = params["features"]
        stratify_on = params["stratify_on"]
        n_fold = params["n_fold"]
        epochs = params["epochs"]
        model = params["model"]

        df["features"] = features
        df["stratification"] = stratify_on
        df["epoch"] = (df["training_iteration"] -1) % epochs
        df["fold"] = (df["training_iteration"] - 1) // epochs
        df["model"] = model

        df_list.append(df)

df_all = pd.concat(df_list)

In [62]:
df_all

,val_score,train_loss,val_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,...,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,features,stratification,epoch,fold,model
0,0.639615,0.366476,0.362870,123.096250,False,NaN,NaN,1,dcb85_00000,a04dbc46cf7d46bc88e93a84afd38aa9,...,131.114.50.210,123.096250,0,1,0.005623,anchor_target_CPCdescription,stratification_index,0,0,distilbert-base-uncased
1,0.710754,0.269216,0.291967,120.830527,False,NaN,NaN,2,dcb85_00000,a04dbc46cf7d46bc88e93a84afd38aa9,...,131.114.50.210,243.926777,0,2,0.005623,anchor_target_CPCdescription,stratification_index,1,0,distilbert-base-uncased
2,0.738008,0.160607,0.265389,118.734818,False,NaN,NaN,3,dcb85_00000,a04dbc46cf7d46bc88e93a84afd38aa9,...,131.114.50.210,362.661595,0,3,0.005623,anchor_target_CPCdescription,stratification_index,2,0,distilbert-base-uncased
3,0.751012,0.118959,0.252512,117.178779,False,NaN,NaN,4,dcb85_00000,a04dbc46cf7d46bc88e93a84afd38aa9,...,131.114.50.210,479.840374,0,4,0.005623,anchor_target_CPCdescription,stratification_index,3,0,distilbert-base-uncased
4,0.756157,0.098471,0.247280,119.298891,False,NaN,NaN,5,dcb85_00000,a04dbc46cf7d46bc88e93a84afd38aa9,...,131.114.50.210,599.139265,0,5,0.005623,anchor_target_CPCdescription,stratification_index,4,0,distilbert-base-uncased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.775561,0.241389,0.227523,236.951253,False,NaN,NaN,16,68096_00000,575f5ac737e54a86ad0559e572e42f3b,...,131.114.50.210,3655.243214,0,16,0.004596,anchor_target_CPCdescription,score_map,0,3,AI-Growth-Lab/PatentSBERTa
16,0.805503,0.137064,0.197761,219.113806,False,NaN,NaN,17,68096_00000,575f5ac737e54a86ad0559e572e42f3b,...,131.114.50.210,3874.357020,0,17,0.004596,anchor_target_CPCdescription,score_map,1,3,AI-Growth-Lab/PatentSBERTa
17,0.810407,0.129559,0.192680,218.703081,False,NaN,NaN,18,68096_00000,575f5ac737e54a86ad0559e572e42f3b,...,131.114.50.210,4093.060102,0,18,0.004596,anchor_target_CPCdescription,score_map,2,3,AI-Growth-Lab/PatentSBERTa
18,0.819756,0.101566,0.182725,219.464206,False,NaN,NaN,19,68096_00000,575f5ac737e54a86ad0559e572e42f3b,...,131.114.50.210,4312.524308,0,19,0.004596,anchor_target_CPCdescription,score_map,3,3,AI-Growth-Lab/PatentSBERTa


- Mean over fold and features comparing models for differing stratification_strategy
- Mean over fold and models comparing features for differing stratification_strategy
- Mean over fold, features and models comparing differing stratification_strategy

In [63]:
df_small = df_all[["val_score", "features", "stratification", "epoch", "fold", "model"]]
df_small

,val_score,features,stratification,epoch,fold,model
0,0.639615,anchor_target_CPCdescription,stratification_index,0,0,distilbert-base-uncased
1,0.710754,anchor_target_CPCdescription,stratification_index,1,0,distilbert-base-uncased
2,0.738008,anchor_target_CPCdescription,stratification_index,2,0,distilbert-base-uncased
3,0.751012,anchor_target_CPCdescription,stratification_index,3,0,distilbert-base-uncased
4,0.756157,anchor_target_CPCdescription,stratification_index,4,0,distilbert-base-uncased
...,...,...,...,...,...,...
15,0.775561,anchor_target_CPCdescription,score_map,0,3,AI-Growth-Lab/PatentSBERTa
16,0.805503,anchor_target_CPCdescription,score_map,1,3,AI-Growth-Lab/PatentSBERTa
17,0.810407,anchor_target_CPCdescription,score_map,2,3,AI-Growth-Lab/PatentSBERTa
18,0.819756,anchor_target_CPCdescription,score_map,3,3,AI-Growth-Lab/PatentSBERTa


In [64]:
df_small[df_small.epoch == 4].groupby(["model", "stratification"]).agg("mean").reset_index()

,model,stratification,val_score,epoch,fold
0,AI-Growth-Lab/PatentSBERTa,score_map,0.823346,4.0,1.5
1,AI-Growth-Lab/PatentSBERTa,stratification_index,0.823402,4.0,1.5
2,distilbert-base-uncased,score_map,0.766991,4.0,1.5
3,distilbert-base-uncased,stratification_index,0.769534,4.0,1.5


In [65]:
roba = df_small[df_small.epoch == 4].groupby(["features", "stratification"]).agg("mean").reset_index()
roba.groupby(["features"]).agg("max").reset_index()

,features,stratification,val_score,epoch,fold
0,CPCdescription_same_anchor_context_similar_tar...,stratification_index,0.801363,4.0,1.5
1,anchor_target_CPCdescription,stratification_index,0.792620,4.0,1.5
2,same_anchor_context_similar_targets,stratification_index,0.795623,4.0,1.5
3,same_anchor_similar_targets,stratification_index,0.796892,4.0,1.5


In [68]:
df_small[df_small.epoch == 4].groupby(["stratification"]).agg("mean").reset_index()

,stratification,val_score,epoch,fold
0,score_map,0.795169,4.0,1.5
1,stratification_index,0.796468,4.0,1.5


In [69]:
df_small[(df_small.epoch == 4) & (df_small.stratification == "stratification_index")].groupby(["features", "model"]).agg("mean").reset_index()

,features,model,val_score,epoch,fold
0,CPCdescription_same_anchor_context_similar_tar...,AI-Growth-Lab/PatentSBERTa,0.829540,4.0,1.5
1,CPCdescription_same_anchor_context_similar_tar...,distilbert-base-uncased,0.773186,4.0,1.5
2,anchor_target_CPCdescription,AI-Growth-Lab/PatentSBERTa,0.820407,4.0,1.5
3,anchor_target_CPCdescription,distilbert-base-uncased,0.763582,4.0,1.5
4,same_anchor_context_similar_targets,AI-Growth-Lab/PatentSBERTa,0.822796,4.0,1.5
5,same_anchor_context_similar_targets,distilbert-base-uncased,0.768449,4.0,1.5
6,same_anchor_similar_targets,AI-Growth-Lab/PatentSBERTa,0.820866,4.0,1.5
7,same_anchor_similar_targets,distilbert-base-uncased,0.772918,4.0,1.5
